In [1]:
import pandas as pd
import numpy as np
from functools import partial
from collections import OrderedDict
from sklearn import manifold
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.linalg import hankel, eigh
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

In [2]:
n_neighbors = 10
n_components = 1
# Set-up manifold methods
LLE = partial(manifold.LocallyLinearEmbedding,
              n_neighbors, n_components, eigen_solver='auto')

methods = OrderedDict()
methods['LLE'] = LLE(method='standard')
methods['LTSA'] = LLE(method='ltsa')
methods['Hessian LLE'] = LLE(method='hessian')
methods['Modified LLE'] = LLE(method='modified')
methods['Isomap'] = manifold.Isomap(n_neighbors, n_components)
methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=1)
methods['SE'] = manifold.SpectralEmbedding(n_components=n_components,
                                           n_neighbors=n_neighbors)
methods['t-SNE'] = manifold.TSNE(n_components=n_components, init='pca',
                                 random_state=0)

In [3]:
df = pd.read_csv('steps_train.txt', skiprows=1,nrows=110, header=None, usecols=[1,2])
df_test = pd.read_csv('steps_test.txt', skiprows=1, nrows=150, header=None, usecols=[1,2])

In [4]:
report_cols = ['Method', 'Training Max Score', 'Training Min Score', 
               'Testing Max Score', 'Testing Min Score', 'Time elapsed']
report = []

In [5]:
def run_algo(r=1, method_name='LLE', L=25):
    
    method=methods[f'{method_name}']
#     print(method_name)
    X_train_data = method.fit_transform(df)
    N = len(X_train_data)
    # L = (N)//2
    L =  25
    X_train = hankel(X_train_data[:L],X_train_data[L-1:]) # Creating trajectory matrix
    eigenValues, eigenVectors = eigh(np.matmul(X_train, X_train.T))
    idx = eigenValues.argsort()[::-1]
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    # Sree plot

    eigen_data = go.Scatter(x=list(range(len(eigenValues[0:10]))), y=eigenValues[0:10])
    eigen_layout = go.Layout(title='ScreePlot', title_x=0.5, xaxis_title="No of eigen values", yaxis_title="eigen value",)
    fig = go.Figure(data=[eigen_data], layout=eigen_layout)
    fig.show()
    
    start = time.perf_counter()
    # Setting statistical dimension based on screeplot above
    r = 2
    # Performing singular value decomposition
    U, Sigma, V = np.linalg.svd(X_train)
    V = V.T
    X_elem = np.array( [Sigma[i] * np.outer(U[:,i], V[:,i]) for i in range(0,r)] )
    X_train_extracted = X_elem.sum(axis=0)
    U = eigenVectors[:,:r] # r as statistical dimension
    UT = U.T
    pX = np.matmul(UT,X_train_extracted)
    centroid = np.mean(pX, axis=1)
    centroid = centroid[:,np.newaxis]
    pXt = np.matmul(UT,X_train)
    dt_matrix = centroid - pXt
    dt_scores = np.linalg.norm(dt_matrix, axis=0, ord=2)

    # Testing positional Deviation
    X_test = method.fit_transform(df_test)
    Xj = hankel(X_test[:L],X_test[L-1:])
    pXj = np.matmul(UT, Xj)
    dj_matrix = centroid - pXj
    dj_scores = np.linalg.norm(dj_matrix, axis=0, ord=2)
    dj_scores = np.asarray(dj_scores)
    end = time.perf_counter()
    finish = time.perf_counter()
#     print("Time Taken for execution is ", round(finish-start, 2), 'Seconds\n\n')
    
    fig = make_subplots(rows=2, cols=2,specs=[[{}, {}], [{"colspan": 2}, None]],
    subplot_titles=("Training plot","Testing plot", f"Anomaly score by {method_name}"))
    fig.add_trace(go.Scatter(x=df[1], y=df[2], marker_color='blue',
                            name="Training path"),row=1, col=1)
    fig.add_trace(go.Scatter(x=df_test[1], y=df_test[2], marker_color='red',
                            name="Testing Score"),row=1, col=2)
    fig.add_trace(go.Scatter(x=list(range(len(dt_scores))), y=dt_scores[5:-5], marker_color='blue',
                             name="Training Score"), row=2, col=1)
    fig.add_trace(go.Scatter(x=list(range(len(dj_scores))), y=dj_scores, marker_color='red',
                            name="Testing Score"), row=2, col=1)
    fig.update_layout(showlegend=False)
    fig.show()
    report.append([method_name, max(dt_scores), min(dt_scores), max(dj_scores), min(dj_scores), round(finish-start, 2)] )
    return (pd.DataFrame(report, columns=report_cols))

In [6]:
methods.keys()

odict_keys(['LLE', 'LTSA', 'Hessian LLE', 'Modified LLE', 'Isomap', 'MDS', 'SE', 't-SNE'])

In [14]:
result = run_algo(method_name='t-SNE')
result

,Method,Training Max Score,Training Min Score,Testing Max Score,Testing Min Score,Time elapsed
0,LLE,0.614756,0.339257,0.552981,0.282888,0.06
1,LTSA,0.510238,0.431487,0.479564,0.261395,0.06
2,Hessian LLE,0.510238,0.431487,0.479564,0.261395,0.04
3,Modified LLE,0.532398,0.405601,0.749987,0.082877,0.04
4,Isomap,64.949332,40.817950,66.737926,23.779329,0.01
5,MDS,47.296974,18.846010,23.926399,4.696800,0.02
6,SE,0.198066,0.119936,0.216395,0.065254,0.02
7,t-SNE,22.362803,2.447675,74.484177,6.866407,0.54


In [18]:
cols = list(result.columns)

In [21]:
cols=['Method',
 'Training Max Score',
 'Testing Max Score',
 'Time elapsed']

In [26]:
result[cols]

,Method,Training Max Score,Testing Max Score,Time elapsed
0,LLE,0.614756,0.552981,0.06
1,LTSA,0.510238,0.479564,0.06
2,Hessian LLE,0.510238,0.479564,0.04
3,Modified LLE,0.532398,0.749987,0.04
4,Isomap,64.949332,66.737926,0.01
5,MDS,47.296974,23.926399,0.02
6,SE,0.198066,0.216395,0.02
7,t-SNE,22.362803,74.484177,0.54


### Diffusion Map based reduction

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances


In [ ]:
df = pd.read_csv('steps_train.txt', skiprows=1,nrows=110, header=None, usecols=[1,2])
df_test = pd.read_csv('steps_test.txt', skiprows=1, nrows=150, header=None, usecols=[1,2])

In [ ]:
def find_diffusion_matrix(X=None, alpha=0.15):
    """Function to find the diffusion matrix P
        
        >Parameters:
        alpha - to be used for gaussian kernel function
        X - feature matrix as numpy array
        
        >Returns:
        P_prime, P, Di, K, D_left
    """
    alpha = alpha
        
    dists = euclidean_distances(X, X)
    K = np.exp(-dists**2 / alpha)
    
    r = np.sum(K, axis=0)
    Di = np.diag(1/r)
    P = np.matmul(Di, K)
    
    D_right = np.diag((r)**0.5)
    D_left = np.diag((r)**-0.5)
    P_prime = np.matmul(D_right, np.matmul(P,D_left))

    return P_prime, P, Di, K, D_left

In [ ]:
def find_diffusion_map(P_prime, D_left, n_eign=3):
    """Function to find the diffusion coordinates in the diffusion space
        
        >Parameters:
        P_prime - Symmetrized version of Diffusion Matrix P
        D_left - D^{-1/2} matrix
        n_eigen - Number of eigen vectors to return. This is effectively 
                    the dimensions to keep in diffusion space.
        
        >Returns:
        Diffusion_map as np.array object
    """   
    n_eign = n_eign
    
    eigenValues, eigenVectors = eigh(P_prime)
    idx = eigenValues.argsort()[::-1]
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    
    diffusion_coordinates = np.matmul(D_left, eigenVectors)
    
    return diffusion_coordinates[:,:n_eign]

In [ ]:
P_prime, P, Di, K, D_left = find_diffusion_matrix(df, alpha=0.15)

In [ ]:
X_train_data = find_diffusion_map(P_prime, D_left, n_eign=1)

In [ ]:
N = len(X_train_data)
# L = (N)//2
L =  50
X_train = hankel(X_train_data[:L],X_train_data[L-1:]) # Creating trajectory matrix
eigenValues, eigenVectors = eigh(np.matmul(X_train, X_train.T))
idx = eigenValues.argsort()[::-1]
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[:,idx]
# Sree plot

eigen_data = go.Scatter(x=list(range(len(eigenValues[0:10]))), y=eigenValues[0:10])
eigen_layout = go.Layout(title='ScreePlot', title_x=0.5, xaxis_title="No of eigen values", yaxis_title="eigen value",)
fig = go.Figure(data=[eigen_data], layout=eigen_layout)
fig.show()

In [ ]:
# Setting statistical dimension based on screeplot above
r = 1
# Performing singular value decomposition
U, Sigma, V = np.linalg.svd(X_train)
V = V.T
X_elem = np.array( [Sigma[i] * np.outer(U[:,i], V[:,i]) for i in range(0,r)] )
X_train_extracted = X_elem.sum(axis=0)
U = eigenVectors[:,:r] # r as statistical dimension
UT = U.T
pX = np.matmul(UT,X_train_extracted)
centroid = np.mean(pX, axis=1)
centroid = centroid[:,np.newaxis]
pXt = np.matmul(UT,X_train)
dt_matrix = centroid - pXt
dt_scores = np.linalg.norm(dt_matrix, axis=0, ord=2)

# Testing positional Deviation
# X_test = method.fit_transform(df_test)
P_prime, P, Di, K, D_left = find_diffusion_matrix(df_test, alpha=0.15)
X_test = find_diffusion_map(P_prime, D_left, n_eign=1)

Xj = hankel(X_test[:L],X_test[L-1:])
pXj = np.matmul(UT, Xj)
dj_matrix = centroid - pXj
dj_scores = np.linalg.norm(dj_matrix, axis=0, ord=2)
dj_scores = np.asarray(dj_scores)

In [ ]:
fig = make_subplots(rows=2, cols=2,specs=[[{}, {}], [{"colspan": 2}, None]],
    subplot_titles=("Training plot","Testing plot", "Anomaly score by Diffusion Map"))
fig.add_trace(go.Scatter(x=df[1], y=df[2], marker_color='blue',
                        name="Training path"),row=1, col=1)
fig.add_trace(go.Scatter(x=df_test[1], y=df_test[2], marker_color='red',
                        name="Testing Score"),row=1, col=2)
fig.add_trace(go.Scatter(x=list(range(len(dt_scores))), y=dt_scores[:], marker_color='blue',
                         name="Training Score"), row=2, col=1)
fig.add_trace(go.Scatter(x=list(range(len(dj_scores))), y=dj_scores, marker_color='red',
                        name="Testing Score"), row=2, col=1)
fig.update_layout(showlegend=False)
